In [84]:
import FinanceDataReader as fdr
import pandas as pd
import numpy as np
import ta
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt

# utils.py
from utils import *

#  차트 설정
%matplotlib inline
import matplotlib.pyplot as plt

plt.rcParams["font.family"] = 'Malgun Gothic'
plt.rcParams["font.family"] = 'AppleGothic'
plt.rcParams["figure.figsize"] = (14,4)
plt.rcParams['lines.linewidth'] = 2
plt.rcParams["axes.grid"] = True

In [220]:
# file = '../../data/krx_2016.csv'
file = '../../data/krx_ta_2016.csv'
# file_path = os.path.join(os.path.dirname(__file__), file)
all_stocks = pd.read_csv(file, index_col=0)


                Name    Code    Open    High     Low   Close    Volume  \
2022-01-27  LG에너지솔루션  373220  597000  598000  450000  505000  15946992   
2022-01-28  LG에너지솔루션  373220  476000  483000  445000  450000   4559773   
2022-02-03  LG에너지솔루션  373220  458000  495500  441000  477000   2918435   
2022-02-04  LG에너지솔루션  373220  476500  505000  476000  504000   2088996   
2022-02-07  LG에너지솔루션  373220  520000  548000  511000  548000   1911176   
...              ...     ...     ...     ...     ...     ...       ...   
2023-11-30    SK바이오팜  326030   84700   86100   84000   86100    133401   
2023-12-01    SK바이오팜  326030   86100   87800   85200   85300     82193   
2023-12-04    SK바이오팜  326030   85300   87800   84500   86300    105264   
2023-12-05    SK바이오팜  326030   85800   91500   85600   91200    230424   
2023-12-06    SK바이오팜  326030   91700   91700   89600   89600    107102   

              Change      ma_5    ma_20  ...      SRSI         TSI         UO  \
2022-01-27       NaN       NaN

In [219]:
feature_columns = df.columns.difference(['Code'])
df = df[feature_columns] 
df = df.replace({0: 0.000001, None: 0.000001}) # 결측치를 매우 작은 수 입력
df = df.dropna(axis=0) # 이동평균선 NaN 행 제거
# df = df.replace({0: 0.000001, None: 0.000001, 'NaN': 0.000001})
pct = df.groupby('Name').pct_change()
pct['Name'] = df['Name']

data = pd.DataFrame()
data = pct.groupby('Name').mean()
data = pd.concat([data, pct.groupby('Name').std()], axis=1)
# data['Volatility'] = pct.groupby('Name').std() 

scaler = StandardScaler().fit(data)
rescaled_dataset = pd.DataFrame(scaler.fit_transform(data), columns=data.columns, index=data.index)


pca_labels = pca_kmeans_clustering(rescaled_dataset,5)
cosine_labels = cosine_kmeans_clustering(rescaled_dataset,5)
cluster_labels = pd.DataFrame()
cluster_labels['pca'] = pca_labels
cluster_labels['cosine'] = cosine_labels
cluster_labels

/Users/kimjuyoung/anaconda3/envs/sesac/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/kimjuyoung/anaconda3/envs/sesac/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,pca,cosine
0,4,4
1,4,4
2,3,0
3,0,1
4,0,1
5,0,1
6,0,1
7,4,4
8,4,1
9,4,4


In [137]:
# Data Transformation
df = all_stocks[['Name', 'Close']].copy()
pct = df.groupby('Name').pct_change()
pct['Name'] = df['Name']

data = pd.DataFrame()
data['Returns'] = pct.groupby('Name').mean() 
data['Volatility'] = pct.groupby('Name').std() 
print(data.head(2))

scaler = StandardScaler().fit(data)
rescaled_dataset = pd.DataFrame(scaler.fit_transform(data), columns=data.columns, index=data.index)
rescaled_dataset

TypeError: Categorical input must be list-like

In [276]:
# 각 회사의 전체 주식 거래 기간의 30% 이상이 Volume이 0인 경우 해당 회사 데이터 삭제
def filter_zero_volume_data(df):
    volume_zero_percentage = df.groupby('Name')['Volume'].apply(lambda x: (x == 0).mean())
    companies_to_remove = volume_zero_percentage[volume_zero_percentage > 0.3].index
    # 조건에 맞는 회사 데이터 삭제
    df_filtered = df[~df['Name'].isin(companies_to_remove)]
    return df_filtered

def standard_scaler(df):
    # 전처리
    df = filter_zero_volume_data(df)
    feature_columns = df.columns.difference(['Code']) # Code 열 제거
    df = df[feature_columns] 
    
    # 결측치 처리
    df = df.replace({0: 0.000001, None: 0.000001}) 
    df = df.dropna(axis=0) # 이동평균선 NaN 행 제거
    # df = df.fillna(0.000001)

    # 변동성을 퍼센트로 변경
    pct = df.groupby('Name').pct_change()
    pct['Name'] = df['Name']

    data = pd.DataFrame()
    data = pct.groupby('Name').mean()
    data = pd.concat([data, pct.groupby('Name').std()], axis=1)

    # scaler
    scaler = StandardScaler().fit(data)
    rescaled_dataset = pd.DataFrame(scaler.fit_transform(data), columns=data.columns, index=data.index)
    return rescaled_dataset

# 코사인 유사도 + K-Means 클러스터링
def cosine_kmeans_clustering(data, n_clusters):
    similarity_matrix = cosine_similarity(data)
    kmeans = KMeans(n_clusters=n_clusters, random_state=42) # random_state : cluster 번호가 변하여 고정
    labels = kmeans.fit_predict(similarity_matrix)
    return labels

# PCA + K-Means 클러스터링
def pca_kmeans_clustering(data, n_clusters):
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(data)
    
    pca = PCA(n_components=2)
    reduced_data = pca.fit_transform(scaled_data)
    
    kmeans = KMeans(n_clusters=n_clusters, random_state=42) # random_state : cluster 번호가 변하여 고정
    labels = kmeans.fit_predict(reduced_data)
    return labels

# 클러스터링 결과 시각화. 개선 필요
def visualize_clusters(data):
    plt.scatter(data.values[:, 0], data.values[:, 1], c=data.values[:, 2], cmap='viridis')
    plt.title('Cluster Visualization')
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.show()

# MultiIndex에서 회사별로 클러스터 라벨 추가
def add_cluster_labels(df, labels):
    df['Cluster'] = labels
    return df

# n개 미만 클러스터 삭제. 이전 버전.
# def filter_clusters(df, n=3):
#     cluster_counts = df['Cluster'].value_counts()
#     clusters_to_keep = cluster_counts[cluster_counts >= n].index
#     df_filtered = df[df['Cluster'].isin(clusters_to_keep)]
#     return df_filtered

# min_count 개 미만 클러스터 제거. 개선 버전
def filter_clusters_with_min_count(cluster_labels, min_count=3):
    for idx_cluster in range(cluster_labels.shape[1]):
        cluster_counts = cluster_labels.iloc[:,idx_cluster].value_counts()
        clusters_to_keep = cluster_counts[cluster_counts >= min_count].index
        cluster_labels = cluster_labels[cluster_labels.iloc[:,idx_cluster].isin(clusters_to_keep)]
    return cluster_labels

# main 함수로 옮겨서 쓸 예정
def make_cluster_labels_by_dataset(df):
    rescaled_dataset = standard_scaler(df)
    pca_labels = pca_kmeans_clustering(rescaled_dataset,5)
    cosine_labels = cosine_kmeans_clustering(rescaled_dataset,5)

    cluster_labels = pd.DataFrame()
    cluster_labels.index = rescaled_dataset.index
    cluster_labels['pca'] = pca_labels
    cluster_labels['cosine'] = cosine_labels
    cluster_labels = filter_clusters_with_min_count(cluster_labels)
    return cluster_labels


# load data
# file = '../../data/krx_2016.csv'
file = '../../data/krx_ta_2016.csv'
# file_path = os.path.join(os.path.dirname(__file__), file)
all_stocks = pd.read_csv(file, index_col=0)
df = all_stocks.copy()
# rescaled_dataset = standard_scaler(df)

# pca_labels = pca_kmeans_clustering(rescaled_dataset,5)
# cosine_labels = cosine_kmeans_clustering(rescaled_dataset,5)
# cluster_labels = pd.DataFrame()
# cluster_labels.index = rescaled_dataset.index
# cluster_labels['pca'] = pca_labels
# cluster_labels['cosine'] = cosine_labels

# cluster_labels = filter_clusters_with_min_count(cluster_labels)
cluster_labels = make_cluster_labels_by_dataset(df)
print(cluster_labels)
# cluster_counts = cluster_labels.iloc[:,0].value_counts()
# # cluster_counts
# clusters_to_keep = cluster_counts[cluster_counts.iloc[:,0] >= 3].index
# clusters_to_keep
# df_filtered = cluster_labels[cluster_labels.isin(clusters_to_keep)]
# df_filtered

          pca  cosine
Name                 
HD한국조선해양    0       3
HD현대중공업     4       0
HMM         4       0
KB금융        2       1
KT          2       1
KT&G        2       1
LG          0       3
LG에너지솔루션    4       0
LG전자        4       2
LG화학        4       2
NAVER       0       3
POSCO홀딩스    4       2
S-Oil       2       4
SK          2       4
SK바이오팜      2       4
SK이노베이션     4       4
SK텔레콤       0       3
SK하이닉스      4       2
고려아연        2       1
기아          4       1
기업은행        2       1
대한항공        4       0
두산에너빌리티     4       0
메리츠금융지주     4       2
삼성SDI       4       2
삼성물산        2       1
삼성생명        2       1
삼성에스디에스     2       1
삼성전기        4       2
삼성전자        4       3
삼성전자우       4       2
삼성중공업       4       0
삼성화재        2       1
셀트리온        4       3
신한지주        2       1
아모레퍼시픽      2       4
우리금융지주      2       1
카카오         0       3
카카오뱅크       2       4
크래프톤        2       4
포스코인터내셔널    4       2
포스코퓨처엠      0       2
하나금융지주      4       1
하이브       

/Users/kimjuyoung/anaconda3/envs/sesac/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/kimjuyoung/anaconda3/envs/sesac/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [272]:
print(cluster_labels.shape[1])
cluster_counts = cluster_labels.iloc[:,0].value_counts()
clusters_to_keep = cluster_counts[cluster_counts >= 3].index
df_filtered = cluster_labels[cluster_labels.iloc[:,0].isin(clusters_to_keep)]


# df_filtered = df_filtered.dropna()
print(df_filtered.shape)
df_filtered

2
(47, 2)


,pca,cosine
Name,,
HD한국조선해양,0,3
HD현대중공업,4,0
HMM,4,0
KB금융,2,1
KT,2,1
KT&G,2,1
LG,0,3
LG에너지솔루션,4,0
LG전자,4,2


In [ ]:
df.dropna(axis=0, inplace=True)

# StandardScaler를 사용하여 데이터 표준화
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df.iloc[:, 1:])

# PCA로 차원 축소
pca = PCA(n_components=2)
reduced_data = pca.fit_transform(scaled_data)

# K-Means 클러스터링
kmeans = KMeans(n_clusters=len(df['Name'].unique()), random_state=42)
labels = kmeans.fit_predict(reduced_data)

# 클러스터링 결과 시각화
df['Cluster'] = labels
colors = plt.cm.Spectral(df['Cluster'].astype(float) / len(df['Cluster'].unique()))
plt.scatter(reduced_data[:, 0], reduced_data[:, 1], c=colors, edgecolor='k', s=40)
plt.title('Cluster Visualization')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')

# 각 점에 회사명 표시
for company, x, y in zip(df['Name'], reduced_data[:, 0], reduced_data[:, 1]):
    plt.text(x, y, company)

plt.show()
print(set(labels))

In [ ]:
# sample
start_date = '20231201'
end_date = '20231205'
code = '005930'
name = '삼성전자'
df = get_dataset(code, name, start_date, end_date)
print(df)

In [ ]:
def get_dataset(stock_code, stock_name, start_date, end_date):
    stock_df = fdr.DataReader(stock_code, start_date, end_date).reset_index()
    stock_df['Name'] = stock_name
    return stock_df
def add_technical_indicators(stock_df):
    ma = [5,20,60,120]
    for days in ma:
        stock_df['ma_'+str(days)] = stock_df['Close'].rolling(window = days).mean()
    H, L, C, V = stock_df['High'], stock_df['Low'], stock_df['Close'], stock_df['Volume']

    stock_df['RSI'] = ta.momentum.rsi(close=C, fillna=True)
    stock_df['MACD'] = ta.trend.macd(close=C, fillna=True)
    stock_df['-VI'] = ta.trend.vortex_indicator_neg(high=H, low=L, close=C, fillna=True)
    stock_df['+VI'] = ta.trend.vortex_indicator_pos(high=H, low=L, close=C, fillna=True)
    return stock_df

# # 샘플 데이터 생성
# start_date = '2023-11-01'
# end_date = '2023-12-05'
# kospi_list = pd.DataFrame({
#     'Code': ['005930', '373220', '000660'],
#     'Name': ['삼성전자', 'LG에너지솔루션', 'SK하이닉스']
# })
start_date = '20231101'
end_date = '20231130'
kospi = fdr.StockListing('KOSPI')
kospi_list = kospi[['Code', 'Name']][:20]

all_stocks = pd.DataFrame()

# 데이터 수집 및 기술적 지표 추가
for code, name in zip(kospi_list['Name'], kospi_list['Code']):
    stock = get_dataset(name, code, start_date, end_date)
    stock = add_technical_indicators(stock)
    all_stocks = pd.concat([all_stocks, stock], ignore_index=True)

# Pivot 데이터 구성
features = all_stocks.columns.drop(['Name', 'Date'])
pivot_stocks = all_stocks.pivot(index='Date', columns='Name', values=features)
pivot_stocks = pivot_stocks.interpolate()

# NaN 값을 0으로 대체
pivot_stocks = pivot_stocks.fillna(0)

# 데이터 정규화
scaler = StandardScaler()
pivot_stocks_scaled = scaler.fit_transform(pivot_stocks)

# PCA를 사용하여 데이터 차원 축소
pca = PCA(n_components=2)
pivot_stocks_pca = pca.fit_transform(pivot_stocks_scaled)

# DBSCAN을 사용한 클러스터링
dbscan = DBSCAN(eps=0.5, min_samples=2)
labels = dbscan.fit_predict(pivot_stocks_pca)

print(labels)
# # 종목별 클러스터 결과 저장
# all_stocks['Cluster'] = labels

# # 종목별 클러스터링 결과 시각화
# for name, group in all_stocks.groupby('Name'):
#     plt.figure(figsize=(8, 5))
#     for cluster_label in set(group['Cluster']):
#         cluster_data = group[group['Cluster'] == cluster_label]
#         plt.scatter(cluster_data['Date'], cluster_data['Close'], label=f'Cluster {cluster_label}')

#     plt.title(f'Clustering Result for {name}')
#     plt.xlabel('Date')
#     plt.ylabel('Close Price')
#     plt.legend()
#     plt.show()


In [ ]:
start_date = '20231201'
end_date = '20231205'

all_stocks = pd.DataFrame()
stock = get_dataset('005930', '삼성전자',start_date, end_date)
stock = add_technical_indicators(stock)
stock1 = get_dataset('373220', 'LG에너지솔루션', start_date, end_date)
stock1 = add_technical_indicators(stock1)
all_stocks = pd.concat([all_stocks, stock], ignore_index=True)
all_stocks = pd.concat([all_stocks, stock1], ignore_index=True)

# for code, name in zip(data_list['Code'], data_list['Name']):
#     tmp = get_dataset(code, start_date, end_date)
#     tmp = add_technical_indicators(tmp)
features = all_stocks.columns.drop(['Name'])

# print(all_stocks.shape)
# print(all_stocks)

pivot_stocks = all_stocks.pivot(index='Date', columns='Name', values=features)
pivot_stocks = pivot_stocks.interpolate()
print(pivot_stocks)
# 데이터 정규화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(pivot_stocks)

# PCA를 사용하여 데이터 차원 축소
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# DBSCAN을 사용한 클러스터링
dbscan = DBSCAN(eps=0.5, min_samples=2)  # eps는 이웃을 결정하는 반경, min_samples는 클러스터가 될 최소 데이터 수
labels = dbscan.fit_predict(X_pca)

# 클러스터링 결과를 시각화
plt.figure(figsize=(10, 6))

for label in set(labels):
    plt.scatter(X_pca[labels == label, 0], X_pca[labels == label, 1], label=f'Cluster {label}')

plt.title('DBSCAN Clustering of Stock Data with Technical Indicators')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend()
plt.show()



In [ ]:
import pandas as pd
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
# 필요한 특징 선택
features = df[['Open', 'High', 'Low', 'Close', 'Volume', 'Change', 'ma_5', 'ma_20', 'ma_60', 'ma_120', 'RSI', 'MACD', '-VI', '+VI']]

# 표준화 (Standardization)
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# DTW 거리 계산 함수
def dtw_distance(series1, series2):
    distance, path = fastdtw(series1, series2, dist=euclidean)
    return distance

# DTW 거리 행렬 생성
dtw_matrix = pd.DataFrame(index=features.index, columns=features.index)
for i in features.index:
    for j in features.index:
        dtw_matrix.at[i, j] = dtw_distance(features_scaled[i], features_scaled[j])

# K-Means 클러스터링
kmeans = KMeans(n_clusters=3, random_state=42)
df['Cluster'] = kmeans.fit_predict(dtw_matrix)

# 클러스터링 결과 시각화
plt.scatter(features['Date'], features['Close'], c=df['Cluster'], cmap='viridis')
plt.title('Stock Clustering based on DTW Distance')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.show()

In [ ]:
# 13개의 회사, 50일치 데이터
num_companies = 13
num_days = 50
num_features = 6

data = np.random.rand(num_companies, num_days, num_features)
columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Change']
companies = [f'Company_{i}' for i in range(num_companies)]

# 임의의 주가 패턴 추가
for i in range(num_companies):
    data[i, :, :] += np.linspace(0, i * 0.1, num_days).reshape(-1, 1)

df = pd.DataFrame(data.reshape(-1, num_features), columns=columns)
df['Company'] = [companies[i] for i in range(num_companies) for _ in range(num_days)]
df['Date'] = np.tile(pd.date_range(start='2022-01-01', periods=num_days), num_companies)

df_pivot = df.pivot(index='Company', columns='Date', values=columns)

print(df_pivot.shape)

In [ ]:
# Calculate average annual percentage return and volatilities over a theoretical one year period
days = df.shape[0]
returns = df.pct_change().mean() * days
returns = pd.DataFrame(returns)
returns.columns = ['Returns']
returns['Volatility'] = df.pct_change().std() * np.sqrt(days)
data=returns
# format the data as a numpy array to feed into the K-Means algorithm
# data = np.asarray([np.asarray(returns['Returns']),np.asarray(returns['Volatility'])]).T 

scaler = StandardScaler().fit(data)
rescaledDataset = pd.DataFrame(scaler.fit_transform(data),columns = data.columns, index = data.index)
# summarize transformed data
rescaledDataset.head(2)
X=rescaledDataset
X.head(2)

In [ ]:
# 기술적 지표를 기반으로 정규화
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)


# DBSCAN for Time Series
dbscan = DBSCAN(metric='euclidean', eps=0.5, min_samples=5)
df['DBSCAN_Labels'] = dbscan.fit_predict(df_scaled)
